# 3. TF-IDF with UMAP

In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial import distance

/Users/meg/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/meg/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/meg/opt/anaconda3/envs/thesis/lib/python3.11/site-packages/

In [2]:
movie_df = pd.read_csv('/Users/meg/Thesis/movie_df_final.csv')
movie_df = movie_df.drop(['Unnamed: 0'], axis=1)
movie_df

,Plot_Words_Len,index,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_improved,Genre_grouped,plt_direc_flg,cast_direc_flg,genre_flg,Genre_19_v1,Default_fixed,Genre_Simple,REGION
0,372,3253,1942,Bugs Bunny Gets the Boid,American,Robert Clampett,NaN,animated short,https://en.wikipedia.org/wiki/Bugs_Bunny_Gets_...,The cartoon begins with a mother buzzard instr...,animation|artisticistic,animation,1.0,NaN,1.0,animation,NaN,animation,American
1,67,6880,1957,This Angry Age,American,Rene Clement,"Silvana Mangano, Anthony Perkins, Richard Conte",drama,https://en.wikipedia.org/wiki/This_Angry_Age,Twenty-year-old and his sixteen-year-old siste...,drama,drama,1.0,1.0,1.0,drama,NaN,drama,American
2,181,32401,2001,Timeranger vs. GoGoV,Japanese,Unknown,NaN,tokusatsu,https://en.wikipedia.org/wiki/Mirai_Sentai_Tim...,"In the 30th century, time travel becomes illeg...",tokusatsu,Default,NaN,NaN,1.0,Default,unknown,unknown,Japanese
3,74,25414,1998,Aunty No. 1,Bollywood,Kirti Kumar,"Govinda, Raveena Tandon, Kader Khan, Harish Kumar",comedy,https://en.wikipedia.org/wiki/Aunty_No._1,is in search of her prince charming and when ...,comedy,comedy,1.0,1.0,1.0,comedy,NaN,comedy,Bollywood
4,693,21147,2014,Dead Snow: Red vs. Dead,British,Director: Tommy Wirkola,"Director: Tommy Wirkola\r\nCast: Vegar Hoel, O...",unknown,https://en.wikipedia.org/wiki/Dead_Snow:_Red_v...,"Previously, and his friends draw the ire of Na...",NaN,Default,1.0,1.0,NaN,Default,unknown,unknown,British
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33515,581,16734,2014,Life After Beth,American,Jeff Baena,Aubrey Plaza\r\nDane DeHaan\r\nAnna Kendrick\r...,comedy,https://en.wikipedia.org/wiki/Life_After_Beth,"After his girlfriend dies, is left devastated....",comedy,comedy,1.0,1.0,1.0,comedy,NaN,comedy,American
33516,522,16304,2012,Goodbye Promise,American,David Branin,"Gregor Collins, Sarah Prikryl, Brian Ronalds","drama, independent movie",https://en.wikipedia.org/wiki/Goodbye_Promise,When moved to Hollywood he made a pact with hi...,drama|independent,drama,1.0,1.0,1.0,drama,NaN,drama,American
33517,458,79,1914,"His Majesty, the Scarecrow of Oz",American,J. Farrell MacDonald,"Violet MacMillan, Pierre Couderc",unknown,"https://en.wikipedia.org/wiki/His_Majesty,_the...",King is a cruel dictator in the Emerald City i...,NaN,Default,1.0,1.0,NaN,Default,unknown,unknown,American
33518,259,12119,1994,Go Fish,American,Rose Troche,Guinevere Turner,drama,https://en.wikipedia.org/wiki/Go_Fish_(film),is a young lesbian student in Chicago who has...,drama,drama,1.0,1.0,1.0,drama,NaN,drama,American


# TF-IDF

reference code: https://makeshiftinsights.com/blog/tf-idf-document-similarity/

In [3]:
vectorizer = TfidfVectorizer(lowercase=True, min_df=20, stop_words='english')
document_term_matrix = vectorizer.fit_transform(movie_df['Plot'])

In [4]:
len(vectorizer.vocabulary_)

15297

In [5]:
document_term_matrix=document_term_matrix.toarray()

In [6]:
document_term_matrix.shape

(33520, 15297)

In [7]:
document_term_matrix_df=pd.DataFrame(document_term_matrix)
document_term_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,15287,15288,15289,15290,15291,15292,15293,15294,15295,15296
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.656664,0.393091,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.076052,0.078636,0.0,0.0,0.0,0.0
33516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
33517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
33518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [8]:
document_term_matrix_df.to_csv('tfidf_embeddings.csv')

### UMAP
reference/source: https://umap-learn.readthedocs.io/en/latest/document_embedding.html

In [9]:
%%time
reducer = umap.UMAP(random_state=42, n_components=200)
tfidfumap = reducer.fit_transform(document_term_matrix)
tfidfumap.shape

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 10min 19s, sys: 13.8 s, total: 10min 33s
Wall time: 5min 36s


(33520, 200)

In [10]:
tfidfumap=pd.DataFrame(tfidfumap)

In [11]:
tfidfumap.to_csv('tfidf_umap_embeddings.csv')